<a href="https://colab.research.google.com/github/agdev/Llamaindex/blob/main/LlamaParser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Installing Librariries**

In [4]:
!pip install llama-index-core -qU
!pip install llama-parse  -qU
!pip install llama-index-readers-file  -qU
!pip install python-dotenv -qU
!pip install llama-parse -qU
!pip install llama-index-embeddings-fastembed -qU
!pip install fastembed -qU
!pip install llama-index-llms-groq -qU
!pip install llama-index-postprocessor-flag-embedding-reranker -qU

In [5]:
!pip install git+https://github.com/FlagOpen/FlagEmbedding.git -qU

  Preparing metadata (setup.py) ... done


In [6]:
from pprint import pprint
import pickle
import os
import subprocess

# **Bring in our LLAMA_CLOUD_API_KEY**

In [7]:
from google.colab import userdata
import nest_asyncio

nest_asyncio.apply()
llamacloud_api_key = userdata.get('LamaCloud')

# **Getting the file/data**

In [8]:
# bring in deps
from llama_parse import LlamaParse
from llama_index.core import SimpleDirectoryReader

MAIN_DIR: str='/content'
file_name_no_ext = "uber_10q_march_2022"
pkl_file_path = os.path.join(MAIN_DIR, f'{file_name_no_ext}.pkl')
pdf_file_path = os.path.join(MAIN_DIR, f'{file_name_no_ext}.pdf')
print(pdf_file_path)
print(pkl_file_path)

pdf_file_exists: bool =os.path.exists(pdf_file_path)
documents = None

if pdf_file_exists:
  print(f'{pdf_file_path} file already exists')
else:
  print(f'{pdf_file_path} file does not exist creating ... ')
  file_name:str = f'{file_name_no_ext}.pdf'
  command:str = f"wget 'https://huggingface.co/silaseic/llamaindex_demo/resolve/main/{file_name}?download=true' -O '{pdf_file_path}'"

  # Run the command
  subprocess.run(command, shell=True, check=True)

  # set up parser
  parser = LlamaParse(api_key=llamacloud_api_key,
      result_type="markdown",  # "markdown" and "text" are available,
      parsing_instruction = """
      You are helpful financial parser
      from following sections:
       - CONDENSED CONSOLIDATED STATEMENTS OF CASH FLOWS
      Extract tables/data under "Results of Operations"
      """
  )

  # use SimpleDirectoryReader to parse our file
  file_extractor = {".pdf": parser}
  documents = SimpleDirectoryReader(input_files=[pdf_file_path], file_extractor=file_extractor).load_data()
  if documents:
      # Save the documents to a file using pickle
      with open(pdf_file_path, 'wb') as f:
          pickle.dump(documents, f)

      from google.colab import files
      files.download(pdf_file_path)
      print(f"Documents saved to {pdf_file_path}")
  else:
      print("Documents are empty. Please check previous steps.")

pkl_file_exists: bool =os.path.exists(pkl_file_path)
if pkl_file_exists:
  print(f'{pkl_file_path} File already exists')
  # Load the documents from the file
  with open(pkl_file_path, 'rb') as f:
      documents = pickle.load(f)
      print(f"Documents loaded from {pkl_file_path}")
else:
  print(f'{pkl_file_path} file does not exist creating ... ')
  if documents:
      # Save the documents to a file using pickle
      with open(pkl_file_path, 'wb') as f:
          pickle.dump(documents, f)

      from google.colab import files
      files.download(pkl_file_path)
      print(f"Documents saved to {pkl_file_path}")
  else:
      print("Documents are empty. Please check previous steps.")


/content/uber_10q_march_2022.pdf
/content/uber_10q_march_2022.pkl
/content/uber_10q_march_2022.pdf file already exists
/content/uber_10q_march_2022.pkl File already exists
Documents loaded from /content/uber_10q_march_2022.pkl


In [9]:
print("len(documents)",len(documents))
pprint(documents)

len(documents) 106
[Document(id_='774ce51e-8bca-4a82-b13b-90b54d8da328', embedding=None, metadata={'file_path': '/content/uber_10q_march_2022.pdf', 'file_name': 'uber_10q_march_2022.pdf', 'file_type': 'application/pdf', 'file_size': 1260185, 'creation_date': '2024-09-11', 'last_modified_date': '2023-10-21'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text="                                                                                UNITED STATES\n                                               SECURITIES AND EXCHANGE COMMISSION\n                                                                                Washington, D.C. 20549\n                                                                  ____________________________________________\n\n          

In [10]:
def group_by_mimetype(documents):
  """Groups documents by their mimetype field.

  Args:
    documents: A list of documents.

  Returns:
    A dictionary where keys are mimetypes and values are lists of documents.
  """
  grouped_documents = {}
  for document in documents:
    mimetype = document.mimetype
    if mimetype not in grouped_documents:
      grouped_documents[mimetype] = []
    grouped_documents[mimetype].append(document)
  return grouped_documents

# Example usage:
grouped_docs = group_by_mimetype(documents)

# Print the grouped documents
for mimetype, docs in grouped_docs.items():
  print(f"Mimetype: {mimetype}")
  print(f"Count: {len(docs)}")
  # for doc in docs:
  #   print(f"  - {doc}")

Mimetype: text/plain
Count: 106


In [25]:
from llama_index.core import VectorStoreIndex
from llama_index.llms.groq import Groq
from llama_index.embeddings.fastembed import FastEmbedEmbedding
from llama_index.core import Settings
GROQ_API_KEY = userdata.get('GROQ_API_KEY')
groq_model_name = "llama3-groq-70b-8192-tool-use-preview" #"mixtral-8x7b-32768"
# by default llamaindex uses OpenAI models
embed_model = FastEmbedEmbedding(model_name="BAAI/bge-base-en-v1.5")

llm = Groq(model=groq_model_name, api_key= GROQ_API_KEY)

Settings.llm = llm
Settings.embed_model = embed_model

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

In [26]:
from llama_index.core.node_parser import MarkdownElementNodeParser

node_parser = MarkdownElementNodeParser(llm = Groq(model=groq_model_name, api_key= GROQ_API_KEY), num_workers=8)

In [27]:
nodes = node_parser.get_nodes_from_documents(documents)

0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
1it [00:00, 1427.61it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
1it [00:00, 6442.86it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
1it [00:00, 7194.35it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?

In [29]:
pprint(nodes)

[TextNode(id_='de658473-5c83-470c-b683-bded3b45fb4b', embedding=None, metadata={'file_path': '/content/uber_10q_march_2022.pdf', 'file_name': 'uber_10q_march_2022.pdf', 'file_type': 'application/pdf', 'file_size': 1260185, 'creation_date': '2024-09-11', 'last_modified_date': '2023-10-21'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='774ce51e-8bca-4a82-b13b-90b54d8da328', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': '/content/uber_10q_march_2022.pdf', 'file_name': 'uber_10q_march_2022.pdf', 'file_type': 'application/pdf', 'file_size': 1260185, 'creation_date': '2024-09-11', 'last_modified_date': '2023-10-21'}, hash='ecb2535bb6acc897b9cf6f4581c6e91ea796115508a068d802ff61a683b1df30')}, text

In [30]:
base_nodes, objects = node_parser.get_nodes_and_objects(nodes)

In [31]:
pprint(base_nodes)

[TextNode(id_='de658473-5c83-470c-b683-bded3b45fb4b', embedding=None, metadata={'file_path': '/content/uber_10q_march_2022.pdf', 'file_name': 'uber_10q_march_2022.pdf', 'file_type': 'application/pdf', 'file_size': 1260185, 'creation_date': '2024-09-11', 'last_modified_date': '2023-10-21'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='774ce51e-8bca-4a82-b13b-90b54d8da328', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': '/content/uber_10q_march_2022.pdf', 'file_name': 'uber_10q_march_2022.pdf', 'file_type': 'application/pdf', 'file_size': 1260185, 'creation_date': '2024-09-11', 'last_modified_date': '2023-10-21'}, hash='ecb2535bb6acc897b9cf6f4581c6e91ea796115508a068d802ff61a683b1df30')}, text

In [32]:
pprint(objects)

[IndexNode(id_='9ea5ebdc-e014-4865-a7a7-b3f518656adb', embedding=None, metadata={'file_path': '/content/uber_10q_march_2022.pdf', 'file_name': 'uber_10q_march_2022.pdf', 'file_type': 'application/pdf', 'file_size': 1260185, 'creation_date': '2024-09-11', 'last_modified_date': '2023-10-21', 'col_schema': 'Column: Redeemable Non-Controlling Interests\nType: number\nSummary: None\n\nColumn: Common Stock\nType: number\nSummary: None\n\nColumn: Additional Paid-In Capital\nType: number\nSummary: None\n\nColumn: Other Comprehensive Income (Loss)\nType: number\nSummary: None\n\nColumn: Accumulated Deficit\nType: number\nSummary: None\n\nColumn: Redeemable Non-Controlling Interests\nType: number\nSummary: None\n\nColumn: Total Equity\nType: number\nSummary: None'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_dat

In [33]:
recursive_index = VectorStoreIndex(nodes=base_nodes+objects)
raw_index = VectorStoreIndex.from_documents(documents)

In [34]:
from llama_index.postprocessor.flag_embedding_reranker import FlagEmbeddingReranker

reranker = FlagEmbeddingReranker(
    top_n=5,
    model="BAAI/bge-reranker-large",
)

recursive_query_engine = recursive_index.as_query_engine(
    similarity_top_k=15,
    node_postprocessors=[reranker],
    verbose=True
)

raw_query_engine = raw_index.as_query_engine(similarity_top_k=15, node_postprocessors=[reranker])

In [35]:
print(len(nodes))

121


## Using `new LlamaParse` as pdf data parsing methods and retrieve tables with two different methods
we compare base query engine vs recursive query engine with tables

### Table Query Task: Queries for Table Question Answering

In [39]:
# query = "how is the Cash paid for Income taxes, net of refunds from Supplemental disclosures of cash flow information?"
# query="""you need to extract tabular data under "CONDENSED CONSOLIDATED STATEMENTS OF CASH FLOW" into Json format example  for each row
# {text: "here goes text of first column",
#  y2021: "here goes value of 2nd column",
#  y2022: "here goes value of 3rd column"}
# """
# query="""
# What was net loss including non-controlling interests?
# """

query="""
# can you extract net loss including non-controlling interests and other items with their resepctive values in the same table?
# """

response_1 = raw_query_engine.query(query)
print("\n***********New LlamaParse+ Basic Query Engine***********")
print(response_1)

response_2 = recursive_query_engine.query(query)
print("\n***********New LlamaParse+ Recursive Retriever Query Engine***********")
print(response_2)



***********New LlamaParse+ Basic Query Engine***********
Net loss including non-controlling interests: $(5,918) million
Other items: $(5,930) million
Retrieval entering 5231a40d-9dec-4c83-846b-02933803701d: TextNode
Retrieving from object TextNode with query 
# can you extract net loss including non-controlling interests and other items with their resepctive values in the same table?
# 
Retrieval entering dc14b95d-a1c9-4292-a89f-b1e06a19ceb1: TextNode
Retrieving from object TextNode with query 
# can you extract net loss including non-controlling interests and other items with their resepctive values in the same table?
# 
Retrieval entering 6586c3c1-10a7-47cf-ad0c-2ae3ee1bd254: TextNode
Retrieving from object TextNode with query 
# can you extract net loss including non-controlling interests and other items with their resepctive values in the same table?
# 
Retrieval entering e2cda3ce-7fd2-4a8e-828c-54c6bd02aa63: TextNode
Retrieving from object TextNode with query 
# can you extract n